In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/darknet'

In [ ]:
import sys
sys.path.append('.')

In [ ]:
!chmod u+x ./darknet

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
import os
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = 15
print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))

#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists('/content/drive/My Drive/Colab Notebooks/darknet/cfg/custom-yolov4-detector.cfg'): os.remove('/content/drive/My Drive/Colab Notebooks/darknet/cfg/custom-yolov4-detector.cfg')


with open('/content/drive/My Drive/Colab Notebooks/darknet/cfg/custom-yolov4-detector.cfg', 'a') as f:
  f.write('[net]' + '\n')
  f.write('batch=64' + '\n')
  #####smaller subdivisions help the GPU run faster. 12 is optimal, but you might need to change to 24,36,64####
  f.write('subdivisions=24' + '\n')
  f.write('width=416' + '\n')
  f.write('height=416' + '\n')
  f.write('channels=3' + '\n')
  f.write('momentum=0.949' + '\n')
  f.write('decay=0.0005' + '\n')
  f.write('angle=0' + '\n')
  f.write('saturation = 1.5' + '\n')
  f.write('exposure = 1.5' + '\n')
  f.write('hue = .1' + '\n')
  f.write('\n')
  f.write('learning_rate=0.001' + '\n')
  f.write('burn_in=1000' + '\n')
  ######you can adjust up and down to change training time#####
  ##Darknet does iterations with batches, not epochs####
  max_batches = num_classes*2000
  #max_batches = 2000
  f.write('max_batches=' + str(max_batches) + '\n')
  f.write('policy=steps' + '\n')
  steps1 = .8 * max_batches
  steps2 = .9 * max_batches
  f.write('steps='+str(steps1)+','+str(steps2) + '\n')

#Instructions from the darknet repo
#change line classes=80 to your number of objects in each of 3 [yolo]-layers:
#change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.

  with open('/content/drive/My Drive/Colab Notebooks/darknet/cfg/yolov4-custom2.cfg', 'r') as f2:
    content = f2.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 0,1,2' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')

  with open('/content/drive/My Drive/Colab Notebooks/darknet/cfg/yolov4-custom3.cfg', 'r') as f3:
    content = f3.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 3,4,5' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')

  with open('/content/drive/My Drive/Colab Notebooks/darknet/cfg/yolov4-custom4.cfg', 'r') as f4:
    content = f4.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 6,7,8' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')
    
  with open('/content/drive/My Drive/Colab Notebooks/darknet/cfg/yolov4-custom5.cfg', 'r') as f5:
    content = f5.readlines()
    for line in content:
      f.write(line)

print("file is written!")    




In [ ]:
!./darknet detector train build/darknet/x64/data/obj_exp.data cfg/custom-yolov4-detector.cfg yolov4.conv.137 -dont_show -map

In [ ]:
import os,glob
import glob
import os

os.chdir(r"/content/drive/My Drive/Colab Notebooks/darknet/build/darknet/x64/data/project2_data/final_img/")
myFiles = glob.glob('*.png')
#print(myFiles)
#path="/content/drive/My Drive/Colab Notebooks/darknet/build/darknet/x64/data/obj"  
#dirList=os.listdir(path)
#f = open("/content/drive/My Drive/Colab Notebooks/darknet/build/darknet/x64/data/obj/output1.txt", "w")
with open("/content/drive/My Drive/Colab Notebooks/darknet/build/darknet/x64/data/project2_data/output2.txt", "w") as f: 
    for filename in myFiles:
        if filename.split('.')[-1]=='png':
            print(filename)
            f.write('./build/darknet/x64/data/project2_data/final_img/'+filename+'\n') 
f.close()

In [ ]:
!./darknet detector calc_anchors build/darknet/x64/data/obj_exp.data -num_of_clusters 9 -width 52 -height 52 -show

In [ ]:
!./darknet detector demo build/darknet/x64/data/obj_exp.data cfg/custom-yolov4-detector.cfg build/darknet/x64/backup/custom-yolov4-detector_best.weights -thresh 0.50 -dont_show build/darknet/x64/Videos/P46_Experiment1_FullRes.mp4 -out_filename res.avi
